In [ ]:
# Cell 1: Import & Konfigurasi
import pandas as pd
import ast
import gc
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Konfigurasi File
INPUT_FILE = 'Data/Data_Siap_Mining.csv' # Sesuaikan path jika perlu
OUTPUT_FILE = 'Final_Skripsi_Rules.csv'

# Konfigurasi Algoritma
MIN_SUPPORT_COUNT = 20
MIN_CONFIDENCE = 0.6

print("✅ Library loaded & Config set.")

# Analisis Anomali Network Menggunakan FP-Growth dan Association Rules

## Deskripsi Proyek
Notebook ini menganalisis log jaringan untuk mendeteksi anomali menggunakan algoritma **FP-Growth** (Frequent Pattern Growth) dan **Association Rules**. Tujuan utama adalah menemukan pola hubungan antara kondisi jaringan dan diagnosis anomali.

## Struktur Analisis
1. **Encoding Data**: Mengonversi data log menjadi format transaksi yang dapat dianalisis
2. **FP-Growth Algorithm**: Menemukan pola frekuen dengan threshold minimum support
3. **Association Rules**: Menghasilkan aturan asosiasi untuk prediksi diagnosis
4. **Filtering & Visualization**: Menyaring aturan yang relevan dan menampilkan hasil akhir

## Tahap 1: Import Library dan Persiapan Data

Pada tahap ini, kami mengimpor library yang diperlukan untuk analisis frequent pattern mining dan memuat dataset yang sudah dibersihkan dari tahap preprocessing.

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
import pandas as pd

# Import cleaned data dari hasil preprocessing
df_cleaned = pd.read_csv('Data/Data_Siap_Mining.csv')
print(f"Dataset berhasil dimuat: {df_cleaned.shape[0]} baris, {df_cleaned.shape[1]} kolom")

## Tahap 2: Encoding Data dan Perhitungan FP-Growth

### Proses Encoding
- **TransactionEncoder**: Mengkonversi data log dari format text menjadi format transaksi (one-hot encoded)
- Setiap item dalam 'items' column diubah menjadi kolom boolean terpisah
- Output: DataFrame `df_encoded` dengan nilai True/False untuk setiap atribut

### Perhitungan Support
- **Minimum Support**: Threshold minimum untuk pola yang dianggap "sering" terjadi
- **MIN_SUPPORT_COUNT = 20**: Pola harus muncul minimal 20 kali dalam dataset
- **min_support_pct**: Persentase support dihitung dari total jumlah transaksi

### Algoritma FP-Growth
- **fpgrowth**: Algoritma efisien untuk mining pola frekuen
- Lebih cepat dari Apriori karena menggunakan FP-tree structure
- Output: `frequent_itemsets` berisi semua pola yang memenuhi threshold

### Association Rules
- Menghasilkan aturan IF-THEN dari pola frekuen
- **min_threshold = 0.6**: Hanya aturan dengan confidence ≥ 60% yang dipertahankan
- Metrik yang dihitung: support, confidence, lift, dsb.

In [ ]:
# Cell 3: Encoding & Mining (REVISI)
# 1. Encoding (Ubah List jadi Matriks Boolean)
print("⏳ Encoding data...")
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

# Hapus list transaksi segera
del transactions
gc.collect()

df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Hapus array numpy segera
del te_ary
gc.collect()

print(f"✅ Encoding Selesai. Shape: {df_encoded.shape}")

# 2. Hitung Threshold Support
min_support_pct = MIN_SUPPORT_COUNT / len(df_encoded)
print(f"🔍 Mining dengan Min Support: {min_support_pct:.5f} (Min {MIN_SUPPORT_COUNT} kejadian)")

# 3. Jalankan FP-Growth (Cari Pola)
frequent_itemsets = fpgrowth(df_encoded, min_support=min_support_pct, use_colnames=True)

# Hapus matriks besar segera (Hemat RAM)
del df_encoded
gc.collect()

print(f"✅ Mining Selesai. Ditemukan {len(frequent_itemsets)} pola.")

In [ ]:
# ===== STEP 2: FP-GROWTH PER DIAGNOSIS LABEL =====
# Jalankan FP-Growth untuk masing-masing diagnosis label
MIN_SUPPORT_COUNT = 20

frequent_itemsets_dict = {}
print(f"✓ Menjalankan FP-Growth per diagnosis label...")
print(f"  - MIN_SUPPORT_COUNT: {MIN_SUPPORT_COUNT}\n")

for label, df_encoded in encoded_data_per_label.items():
    print(f"FP-Growth for '{label}'...", end=" ")
    
    # Hitung minimum support percentage
    min_support_pct = MIN_SUPPORT_COUNT / len(df_encoded)
    
    # Gunakan hanya kolom atribut (exclude diagnosis column)
    df_for_fpgrowth = df_encoded.drop('diagnosis', axis=1)
    
    # Jalankan FP-Growth
    frequent_itemsets = fpgrowth(df_for_fpgrowth, min_support=min_support_pct, use_colnames=True)
    
    # Tambahkan diagnosis label ke dalam frequent itemsets
    frequent_itemsets['diagnosis'] = label
    
    frequent_itemsets_dict[label] = frequent_itemsets
    
    print(f"✓ Ditemukan {len(frequent_itemsets)} pola frekuen")
    
    # Bersihkan memori
    del df_encoded, df_for_fpgrowth
    gc.collect()

print(f"\n✓ FP-Growth selesai untuk semua labels!")

# Gabungkan semua frequent itemsets
all_frequent_itemsets = pd.concat(frequent_itemsets_dict.values(), ignore_index=True)
print(f"  - Total pola frekuen: {len(all_frequent_itemsets)}")


In [ ]:
# ===== STEP 3: GENERATE ASSOCIATION RULES PER LABEL =====
# Buat association rules untuk setiap diagnosis label
MIN_CONFIDENCE = 0.6

rules_dict = {}
total_rules = 0

print(f"✓ Menghasilkan Association Rules per diagnosis label...")
print(f"  - MIN_CONFIDENCE: {MIN_CONFIDENCE}\n")

for label, frequent_itemsets in frequent_itemsets_dict.items():
    print(f"Generating rules for '{label}'...", end=" ")
    
    if len(frequent_itemsets) == 0:
        print("(Skip - tidak ada frequent itemsets)")
        continue
    
    # Generate association rules
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=MIN_CONFIDENCE)
    
    if len(rules) > 0:
        rules['diagnosis'] = label
        rules_dict[label] = rules
        total_rules += len(rules)
        print(f"✓ Ditemukan {len(rules)} rules")
    else:
        print("(tidak ada rules dengan confidence >= threshold)")
    
    # Bersihkan memori
    del frequent_itemsets
    gc.collect()

print(f"\n✓ Association Rules generation selesai!")
print(f"  - Total rules: {total_rules}")

# Gabungkan semua rules
if rules_dict:
    all_rules = pd.concat(rules_dict.values(), ignore_index=True)
    print(f"  - Rules yang digabungkan: {len(all_rules)}")
else:
    all_rules = pd.DataFrame()
    print("  - Tidak ada rules yang dihasilkan!")


## Tahap 3: Penyaringan dan Tampilan Hasil Akhir

### Konsep Filtering
Dari semua association rules yang dihasilkan, kita hanya ingin menyimpan rules yang:
- **Consequent (Kesimpulan)**: Harus berupa diagnosis/label anomali
  - NORMAL: Kondisi jaringan normal
  - UPSTREAM_FAILURE: Kegagalan uplink
  - LINK_FAILURE: Kegagalan link
  - DDOS_ATTACK: Serangan DDoS
  - BROADCAST_STORM: Broadcast storm

Contoh Rule yang BENAR (yang kita inginkan):
- IF `ether1=down AND queue=high` THEN `LINK_FAILURE` ✓

Contoh Rule yang SALAH (yang kita buang):
- IF `LINK_FAILURE` THEN `ether1=down` ✗

### Metrik Evaluasi
- **Support**: Seberapa sering pola ini muncul dalam dataset (0-1)
- **Confidence**: Probabilitas consequent terjadi jika antecedent terjadi (0-1)
- **Lift**: Seberapa kuat hubungan antara antecedent dan consequent (>1 = hubungan positif)

In [ ]:
# ===== STEP 4: FILTERING RULES =====
# Filter rules - hanya ambil yang berakhir dengan diagnosis
def is_diagnosis(consequents):
    """
    Cek apakah consequents (frozenset) mengandung minimal 1 label diagnosis
    """
    return any(label in consequents for label in DIAGNOSIS_LABELS)

if len(all_rules) > 0:
    final_rules = all_rules[all_rules['consequents'].apply(is_diagnosis)].copy()
    print(f"✓ Rules setelah filtering:")
    print(f"  - Total rules sebelum filter: {len(all_rules)}")
    print(f"  - Rules dengan diagnosis consequent: {len(final_rules)}")
    print(f"  - Rules yang dihilangkan: {len(all_rules) - len(final_rules)}")
    
    # ===== STEP 5: SORTING =====
    # Urutkan berdasarkan confidence (tertinggi) kemudian support (tertinggi)
    final_rules = final_rules.sort_values(['confidence', 'support'], ascending=[False, False])
    print(f"\n✓ Rules sudah diurutkan berdasarkan confidence dan support (descending)")
    
    # ===== STEP 6: DISPLAY HASIL =====
    # Konversi frozenset ke list untuk memudahkan pembacaan CSV dan Streamlit
    final_rules_for_display = final_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift', 'diagnosis']].copy()
    final_rules_for_display['antecedents'] = final_rules_for_display['antecedents'].apply(list)
    final_rules_for_display['consequents'] = final_rules_for_display['consequents'].apply(list)
    
    print(f"\n{'='*100}")
    print(f"TOP ASSOCIATION RULES - NETWORK ANOMALY DETECTION (PROCESSED PER LABEL)")
    print(f"{'='*100}\n")
    
    # Tampilkan dengan format yang lebih rapi (hanya 15 untuk display)
    for idx, (i, row) in enumerate(final_rules_for_display.head(15).iterrows(), 1):
        antecedents = ', '.join(row['antecedents'])
        consequents = ', '.join(row['consequents'])
        diagnosis = row['diagnosis']
        
        print(f"[Rule {idx}] [{diagnosis}]")
        print(f"  IF:   {antecedents}")
        print(f"  THEN: {consequents}")
        print(f"  Support: {row['support']:.4f} | Confidence: {row['confidence']:.4f} | Lift: {row['lift']:.4f}")
        print()
    
    # ===== STEP 7: SAVE HASIL =====
    print(f"{'='*100}")
    print(f"MENYIMPAN DATA KE CSV...")
    print(f"{'='*100}\n")
    
    # Simpan SEMUA final rules ke CSV
    final_rules_for_display.to_csv('../Final_Skripsi_Rules.csv', index=False)
    print(f"✓ SEMUA {len(final_rules_for_display)} rules berhasil disimpan ke 'Final_Skripsi_Rules.csv'")
    print(f"  - File ini akan digunakan oleh Dashboard Live Monitoring")
    print(f"  - Frozenset sudah dikonversi ke list untuk kompatibilitas Streamlit")
    print(f"  - Format: antecedents | consequents | support | confidence | lift | diagnosis")
    
    print(f"\n{'='*100}")
    print(f"✓ TOP 15 RULES (sample dari {len(final_rules_for_display)} total rules)")
    print(f"{'='*100}")
    
    # Bersihkan memori
    del all_rules, all_frequent_itemsets
    gc.collect()
    
else:
    print("⚠ Tidak ada rules yang dihasilkan dari semua labels!")


## Kesimpulan dan Interpretasi

### Hasil Analisis
Notebook ini menghasilkan association rules yang menunjukkan hubungan antara kondisi jaringan dan tipe anomali. Setiap rule memiliki:
- **Antecedents (IF)**: Kondisi atau kombinasi kondisi jaringan yang diamati
- **Consequents (THEN)**: Diagnosis/prediksi tipe anomali yang kemungkinan terjadi
- **Confidence**: Tingkat akurasi prediksi (semakin tinggi semakin baik)
- **Support**: Seberapa sering pola ini muncul dalam data (frekuensi)
- **Lift**: Seberapa kuat hubungan antara kondisi dan diagnosis

### Penggunaan Praktis
Hasil rules ini dapat digunakan untuk:
1. **Early Warning System**: Mendeteksi anomali sebelum terjadi dengan mengecek kondisi antecedent
2. **Root Cause Analysis**: Memahami faktor-faktor apa yang menyebabkan setiap tipe anomali
3. **Network Optimization**: Meningkatkan monitoring dan maintenance protokol berdasarkan insights
4. **Decision Support**: Membantu network administrator membuat keputusan yang lebih cepat dan tepat

### File Output
- **Final_Skripsi_Rules.csv**: Berisi top 10 rules dalam format CSV yang dapat digunakan untuk proses selanjutnya

In [ ]:
# Cell 2: Load Data & Gabung Label (REVISI)
import pandas as pd
import ast
import gc
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Konfigurasi
INPUT_FILE = 'Data/Data_Siap_Mining.csv'
OUTPUT_FILE = 'Final_Skripsi_Rules.csv'
MIN_SUPPORT_COUNT = 20
MIN_CONFIDENCE = 0.6

# 1. Load Data
df = pd.read_csv(INPUT_FILE)
print(f"📊 Dataset dimuat: {len(df)} baris.")

# 2. Deteksi Nama Kolom Label (diagnosis atau Label)
label_col = 'Label' if 'Label' in df.columns else 'diagnosis'
print(f"ℹ️ Kolom Target: {label_col}")

# 3. GABUNGKAN Gejala + Label (KUNCI UTAMA AGAR BERHASIL)
# Kita memasukkan Label Diagnosis ke dalam keranjang belanja yang sama dengan gejalanya.
transactions = []
for _, row in df.iterrows():
    # Parse string list kembali ke list python
    items = ast.literal_eval(row['items']) if isinstance(row['items'], str) else row['items']
    # Tambahkan Label Diagnosis ke dalam list items
    items.append(row[label_col]) 
    transactions.append(items)

# Hapus DF awal buat hemat RAM
del df
gc.collect()

print(f"✅ Transaksi Siap (Contoh: {transactions[0]})")

In [ ]:
# Cell 4: Generate Rules & Filtering (REVISI)
# 1. Generate Rules (Cari Hubungan Sebab-Akibat)
print("⚙️ Generating Rules...")
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=MIN_CONFIDENCE)

del frequent_itemsets
gc.collect()

# 2. Filtering (Hanya Simpan Rule yang Ujungnya Diagnosis)
TARGET_LABELS = ['NORMAL', 'UPSTREAM_FAILURE', 'LINK_FAILURE', 'DDOS_ATTACK', 'BROADCAST_STORM']

def is_diagnosis_rule(consequents):
    return any(label in consequents for label in TARGET_LABELS)

final_rules = rules[rules['consequents'].apply(is_diagnosis_rule)].copy()

# 3. Formatting untuk CSV
# Ubah format frozenset ({'a'}) jadi list biasa ['a'] biar bisa dibaca Dashboard
final_rules['antecedents'] = final_rules['antecedents'].apply(lambda x: list(x))
final_rules['consequents'] = final_rules['consequents'].apply(lambda x: list(x))

# Sort dari yang paling akurat
final_rules = final_rules.sort_values(['confidence', 'support'], ascending=[False, False])

# 4. Simpan ke CSV
final_rules.to_csv(OUTPUT_FILE, index=False)

print(f"\n{'='*50}")
print(f"✅ SUKSES! Total Rules Valid: {len(final_rules)}")
print(f"💾 File Rules tersimpan di: {OUTPUT_FILE}")
print(f"{'='*50}")

# Preview Top 5
print("\nTop 5 Rules Terbaik:")
for idx, row in final_rules.head(5).iterrows():
    print(f"IF {row['antecedents']} THEN {row['consequents']} (Conf: {row['confidence']:.2f})")

In [ ]:
# Cell 4: FP-Growth Mining
# 1. Hitung Support Percentage
min_support_pct = MIN_SUPPORT_COUNT / len(df_encoded)
print(f"🔍 Mining dengan Min Support: {min_support_pct:.5f} (Setara {MIN_SUPPORT_COUNT} baris)")

# 2. Jalankan FP-Growth
frequent_itemsets = fpgrowth(df_encoded, min_support=min_support_pct, use_colnames=True)

# 3. Hapus matriks besar secepatnya (Paling makan RAM)
del df_encoded
gc.collect()

print(f"✅ Mining selesai. Ditemukan {len(frequent_itemsets)} pola itemset.")

In [ ]:
# Cell 5: Generate Rules & Filtering (Final)
# 1. Generate Rules
print("⚙️ Sedang generate Association Rules...")
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=MIN_CONFIDENCE)

# 2. Hapus itemsets
del frequent_itemsets
gc.collect()

# 3. Filtering: Hanya ambil rule yang berujung DIAGNOSIS
TARGET_LABELS = ['NORMAL', 'UPSTREAM_FAILURE', 'LINK_FAILURE', 'DDOS_ATTACK', 'BROADCAST_STORM']

def is_diagnosis_rule(consequents):
    # Cek apakah hasil (THEN) adalah salah satu label diagnosis
    return any(label in consequents for label in TARGET_LABELS)

# Terapkan filter
final_rules = rules[rules['consequents'].apply(is_diagnosis_rule)].copy()

# 4. Sorting & Formatting
final_rules = final_rules.sort_values(['confidence', 'support'], ascending=[False, False])

# Ubah frozenset jadi list biasa agar bersih di CSV
final_rules['antecedents'] = final_rules['antecedents'].apply(lambda x: list(x))
final_rules['consequents'] = final_rules['consequents'].apply(lambda x: list(x))

# 5. Simpan
final_rules.to_csv(OUTPUT_FILE, index=False)

print(f"\n{'='*50}")
print(f"✅ SUKSES! Total Rules Valid: {len(final_rules)}")
print(f"💾 Disimpan ke: {OUTPUT_FILE}")
print(f"{'='*50}")

# Preview Top 5
print("\nTop 5 Rules Terbaik:")
for idx, row in final_rules.head(5).iterrows():
    print(f"IF {row['antecedents']} THEN {row['consequents']} (Conf: {row['confidence']:.2f})")